剑指 Offer 41. 数据流中的中位数

如何得到一个数据流中的中位数？如果从数据流中读出奇数个数值，那么中位数就是所有数值排序之后位于中间的数值。如果从数据流中读出偶数个数值，那么中位数就是所有数值排序之后中间两个数的平均值。

例如，
[2,3,4] 的中位数是 3
[2,3] 的中位数是 (2 + 3) / 2 = 2.5

设计一个支持以下两种操作的数据结构：
void addNum(int num) - 从数据流中添加一个整数到数据结构中。
double findMedian() - 返回目前所有元素的中位数。

示例 1：
输入：
["MedianFinder","addNum","addNum","findMedian","addNum","findMedian"]
[[],[1],[2],[],[3],[]]
输出：[null,null,null,1.50000,null,2.00000]

示例 2：
输入：
["MedianFinder","addNum","findMedian","addNum","findMedian"]
[[],[2],[],[3],[]]
输出：[null,null,2.00000,null,2.50000]

In [ ]:
#给定一长度为N的无序数组，其中位数的计算方法：首先对数组执行排序（使用 O(Nlog N)时间），然后返回中间元素即可（使用O(1)时间）。
#根据以上思路，可以将数据流保存在一个列表中，并在添加元素时保持数组有序 。
#此方法的时间复杂度为O(N)，其中包括：查找元素插入位置O(logN)（二分查找）、向数组某位置插入元素O(N)（插入位置之后的元素都需要向后移动一位）。
#借助堆可进一步优化时间复杂度。

In [ ]:
#建立一个小顶堆A和大顶堆B，各保存列表的一半元素，且规定：
#A保存较大的一半，am …… a3，a2，a1（堆顶）
#长度为N/2（N为偶数）或者（N+1）/2（N为奇数）
#B保存较小的一半，b1（堆顶），b2，b3 …… bn
#长度为N/2（N为偶数）或者（N-1）/2（N为奇数）
#N=m+n，则中位数=a1（N为偶数）或者（a1+b1）/2（N为奇数）
#A的数量大于等于B的数量

#addNum(num) 函数：
#当m=n（即N为偶数）：需向A添加一个元素。实现方法：将新元素num插入至B，再将B堆顶元素插入至A
#--------⚠️此处是我和大佬的分界线----------
#我们只用m和n的和来判断向哪个数组加num，所以这里num可能属于 “较小的一半”（即属于B），因此不能将num直接插入至A！
#而应先将num插入至B，再将B堆顶元素插入至A。
#这是因为：1.如果num小于B的堆顶，那么num老老实实呆在B中间的地方，堆顶还是最大，因为要保持数量所以被插入到A
#        2.如果num大于B的堆顶（本身确实应该进入A），那么num成为了新堆顶，“再将B堆顶元素插入至A”这个操作让他曲线救国最终还是进了A！
#这样就可以始终保持A保存较大一半、B保存较小一半～～～
#当m!=n（即N为奇数）：需向B添加一个元素。实现方法：将新元素num插入至A，再将A堆顶元素插入至B

#findMedian() 函数：
#当m=m（即N为偶数）：则中位数为（A的堆顶元素+B的堆顶元素）/2
#当m!=n（即N为奇数）：中位数为A的堆顶元素

In [ ]:
#要判断进来的数字进入哪个堆
#进入堆后自动排序
#判断现在N是偶数还是奇数
from heapq import *
class MedianFinder:
    def __init__(self):
        self.A = [] # 小顶堆，保存较大的一半
        self.B = [] # 大顶堆，保存较小的一半
        #A和B都是数组

    def addNum(self, num):
        if len(self.A) != len(self.B):#m!=n（即N为奇数）：需向B添加一个元素
            heappush(self.A, num)#实现方法：将新元素num插入至A
            heappush(self.B, -heappop(self.A))#再将A堆顶元素插入至B
        else:#当m=n（即N为偶数）：需向A添加一个元素
            heappush(self.B, -num)#实现方法：将新元素num插入至B
            heappush(self.A, -heappop(self.B))#再将B堆顶元素插入至A
        #headpush自动排序

    def findMedian(self):
        if len(self.A) != len(self.B):
            return self.A[0]#中位数=a1（N为偶数）,A[0]是堆顶
        else:
            return (self.A[0] - self.B[0]) / 2.0#中位数=（a1+b1）/2（N为奇数）